In [ ]:
# Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Set company ticker and peers for comparison
company_ticker = 'MARUTI.NS'  # Maruti Suzuki India Limited
peers = ['TATAMOTORS.NS', 'M&M.NS', 'HYUNDAI.BO']  # Peers for relative valuation

# Download financial data from Yahoo Finance
# Step 1: Download financial data (Income Statement, Balance Sheet, and Cash Flow)
company = yf.Ticker(company_ticker)
balance_sheet = company.balance_sheet
income_statement = company.financials
cash_flow = company.cashflow

# Display the first few rows of each financial statement
print(balance_sheet.head())
print(income_statement.head())
print(cash_flow.head())

# Step 2: Perform common sizing and ratio analysis for Income Statement and Balance Sheet

# Common sizing for Income Statement (all items as % of total revenue)
income_common_size = income_statement.div(income_statement.loc['Total Revenue'])
print(income_common_size.head())

# Common sizing for Balance Sheet (all items as % of total assets)
balance_common_size = balance_sheet.div(balance_sheet.loc['Total Assets'])
print(balance_common_size.head())

# Step 3: Horizontal Analysis (year-over-year percentage changes)
income_horizontal = income_statement.pct_change(axis='columns') * 100
balance_horizontal = balance_sheet.pct_change(axis='columns') * 100

print(income_horizontal.head())
print(balance_horizontal.head())

# Step 4: Calculate financial ratios
# Example: Profitability Ratios, Liquidity Ratios, and Solvency Ratios
# Gross Profit Margin, Net Profit Margin, Current Ratio, Debt-to-Equity Ratio
gross_profit_margin = income_statement.loc['Gross Profit'] / income_statement.loc['Total Revenue']
net_profit_margin = income_statement.loc['Net Income'] / income_statement.loc['Total Revenue']
current_ratio = balance_sheet.loc['Total Current Assets'] / balance_sheet.loc['Total Current Liabilities']
debt_to_equity = balance_sheet.loc['Total Debt'] / balance_sheet.loc['Total Stockholder Equity']

# Output the calculated ratios
print("Gross Profit Margin:\n", gross_profit_margin)
print("Net Profit Margin:\n", net_profit_margin)
print("Current Ratio:\n", current_ratio)
print("Debt to Equity Ratio:\n", debt_to_equity)

# Step 5: Forecast Free Cash Flow (FCFF and FCFE)
# Assuming a 5-year planning horizon based on historical data and ratio analysis

# Function to calculate FCFF
def calculate_fcff(cashflow, operating_income, capex, working_capital_change):
    fcff = operating_income - capex - working_capital_change
    return fcff

# Define past values for example (real values should be extracted from financials)
operating_income = cash_flow.loc['Operating Cash Flow']
capex = cash_flow.loc['Capital Expenditures']
working_capital_change = cash_flow.loc['Change In Working Capital']

# Forecast FCFF for the next 5 years
fcff_forecast = []
for year in range(5):
    fcff_year = calculate_fcff(cash_flow, operating_income, capex, working_capital_change)
    fcff_forecast.append(fcff_year * (1 + 0.05))  # Assuming 5% growth

print("FCFF Forecast for next 5 years:", fcff_forecast)

# Step 6: Perform Discounted Cash Flow (DCF) Valuation
# Assume a discount rate (WACC) and calculate the terminal value

# Discount rate (WACC)
discount_rate = 0.1  # 10%, you can calculate this based on the CAPM model

# Terminal Value using perpetuity formula
terminal_value = fcff_forecast[-1] * (1 + 0.03) / (discount_rate - 0.03)

# Calculate the present value of FCFFs
fcff_pv = []
for i in range(len(fcff_forecast)):
    fcff_pv.append(fcff_forecast[i] / (1 + discount_rate) ** (i + 1))

# Sum of present value of FCFFs + Terminal value
enterprise_value = sum(fcff_pv) + terminal_value
print("Enterprise Value (DCF):", enterprise_value)

# Step 7: Relative Valuation using Comparable Companies
# Download financial data for peer companies
peer_data = {}
for peer in peers:
    peer_data[peer] = yf.Ticker(peer).financials

# Calculate multiples (e.g., P/E, EV/EBITDA) for peers and Maruti Suzuki
def calculate_pe_ratio(financials):
    return financials.loc['Net Income'] / financials.loc['Shares Outstanding']

# For example, calculating Price-to-Earnings (P/E) ratio for Maruti Suzuki and its peers
pe_ratios = {}
for peer in peers:
    pe_ratios[peer] = calculate_pe_ratio(peer_data[peer])

# Get Maruti Suzuki P/E ratio
maruti_pe_ratio = calculate_pe_ratio(income_statement)

print("P/E Ratios:\n", pe_ratios)
print("Maruti P/E Ratio:", maruti_pe_ratio)

# Step 8: Industry Analysis
# Perform additional industry analysis using reports from sources like IBEF

# Final Deliverables: Compile the data into a presentation format
# You can use Python libraries like matplotlib for plots and pandas for data tables in your report

# Example: Plot FCFF forecast
years = [2025, 2026, 2027, 2028, 2029]
plt.plot(years, fcff_forecast, label="FCFF Forecast")
plt.title('Free Cash Flow to Firm (FCFF) Forecast')
plt.xlabel('Year')
plt.ylabel('FCFF (in billions)')
plt.legend()
plt.show()

